## Importando bibliotecas

In [1]:
import re
import json

In [2]:
data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [3]:
print(data[0].keys())

dict_keys(['title', 'body'])


## Função para encontrar links HTML

In [4]:
# Baseado em https://gist.github.com/gruber/249502
def rm_htmlLinks(texto):
    return re.sub(r"(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))", "", texto)

## Função para limpar os wikilinks

In [5]:
def sub_links(texto):
    return re.sub(r"\[\[(?:[^|]*?\|)*?([^|]*?)\]\]", r"\1", texto)

## Função para limpar os templates

In [6]:
def rm_templates(texto):
    count = 0
    spansToRemove = []
    for item in re.finditer(r"{{|}}", texto):
        if item[0] == "{{":
            if count == 0:
                begin = item.span()[0]
            count += 1
        else:
            count -= 1
            if count == 0:
                end = item.span()[1]
                spansToRemove.append((begin, end))
    cleanText = ""
    begin = 0
    for span in spansToRemove:
        end, nextBegin = span
        cleanText += texto[begin:end]
        begin = nextBegin
    cleanText += texto[begin:]
    return cleanText

In [7]:
def rm_templates2(texto):
    count = 0
    spansToRemove = []
    for item in re.finditer(r"{[\|]|[\|]}", texto):
        if item[0] == "{|":
            if count == 0:
                begin = item.span()[0]
            count += 1
        else:
            count -= 1
            if count == 0:
                end = item.span()[1]
                spansToRemove.append((begin, end))
    cleanText = ""
    begin = 0
    for span in spansToRemove:
        end, nextBegin = span
        cleanText += texto[begin:end]
        begin = nextBegin
    cleanText += texto[begin:]
    return cleanText

## Função para remover as as ocorrências de ```<ref>...</ref>```

In [8]:
def rm_refs(texto):
    return re.sub(r"<ref.*>.*</ref>|<ref.*>|</ref>", "", texto)

In [9]:
def rm_htmlTags(texto):
    return re.sub(r"<.*?>|</.*?>", "", texto)

## Função para remover as ocorrências de aspas (simples ou dupla)

In [10]:
def rm_aspas(texto):
    return re.sub(r"(['\"]+)(.*?)\1", r"\2", texto)

In [11]:
def rm_caracterEspecial(texto):
    sem_travessao = re.sub(r"\s\-\s|\-\-+", " ", texto)
    return re.sub(r"[^\w\s\-]", " ", sem_travessao)

In [12]:
def rm_digitos(texto):
    return re.sub(r"\w*\d\w*", "", texto)

In [13]:
def rm_travessaoErrado(texto):
    return re.sub(r"(\w+)\-|\-(\w+)", r"\1\2", texto)

## Pipeline Completa

In [14]:
def limpa_textos(txt):
    output = rm_htmlLinks(txt)
    output = sub_links(output)
    output = rm_refs(output)
    output = rm_htmlTags(output)
    output = rm_templates(output)
    output = rm_templates2(output)
    output = rm_aspas(output)
    output = rm_caracterEspecial(output)
    output = rm_digitos(output)
    output = rm_travessaoErrado(output)
    return output

In [19]:
print(data[1313]["body"])

{{Evento atual|tema=construção}}
[[Imagem:NicaraguaCanalPanorama.jpg|thumb|Concepção artística do canal proposto, 1899]]
[[Imagem:1895NicaraguaCanalCartoon.jpg|thumb|Cartum de 1895 defendendo a ação dos EUA para a construção do Canal da Nicarágua.]]
O '''Canal da Nicarágua''' (espanhol: ''Canal de Nicaragua''), formalmente o '''Canal''' da '''Nicarágua e Projeto de Desenvolvimento''' (também conhecido como '''Grande Canal''' da '''Nicarágua''', ou '''Grande Canal Interoceânico''' foi uma rota de navegação proposta através da Nicarágua para conectar o [[Mar do Caribe]] (e, portanto, o [[Oceano Atlântico]]) com o [[Oceano Pacífico]]. Os cientistas estavam preocupados com o impacto ambiental do projeto, já que o Lago Nicarágua é o principal reservatório de água doce da América Central enquanto a viabilidade do projeto foi questionada por engenheiros e especialistas em navegação.<ref name=":0">{{citar web |ultimo= |primeiro= |url=https://www.bloomberg.com/news/articles/2016-03-17/china-slo

In [18]:
print(limpa_textos(data[1313]["body"]))


Concepção artística do canal proposto  
Cartum de  defendendo a ação dos EUA para a construção do Canal da Nicarágua 
O Canal da Nicarágua  espanhol  Canal de Nicaragua   formalmente o Canal da Nicarágua e Projeto de Desenvolvimento  também conhecido como Grande Canal da Nicarágua  ou Grande Canal Interoceânico foi uma rota de navegação proposta através da Nicarágua para conectar o Mar do Caribe  e  portanto  o Oceano Atlântico  com o Oceano Pacífico  Os cientistas estavam preocupados com o impacto ambiental do projeto  já que o Lago Nicarágua é o principal reservatório de água doce da América Central enquanto a viabilidade do projeto foi questionada por engenheiros e especialistas em navegação  

A construção de um canal usando o Rio San Juan como via de acesso ao Lago Nicarágua foi proposta pela primeira vez no início da era colonial  Os Estados Unidos abandonaram os planos de construir uma hidrovia na Nicarágua no início do século   depois que compraram as participações francesas n